# BM25-CTF Algorithm

In [33]:
import pickle
def savePickle(dictfile,filename):
	print("saving....")
	f=open(filename+".pickle",'wb')
	pickle.dump(dictfile,f)
	f.close()

def loadPickle(filename):
	print("loading....")
	f=open(filename+".pickle",'rb')
	dictfile=pickle.load(f)
	f.close()
	return dictfile

In [34]:
meta=loadPickle('meta')

loading....


In [85]:
import math

class bm25_ctf():                                            
                                                                       
    def __init__(self, k1 = 1.25, b = 0.9, k3 = 1000):                                             
        self.k1 = k1
        self.b = b
        self.k3 = k3                                      
                                                                                 
    def score_one(self, w,query_meta,meta,document_id):
        k1 = self.k1
        b = self.b
        k3 = self.k3
        #Fill your answer here
        ictf=math.log(meta['total_terms']/meta['corpus_term_count'][w])
        pidf=math.log(-(1-2**(-meta['corpus_term_count'][w]/meta['num_docs']))/(math.log(1-meta['doc_count'][w]/meta['num_docs']))+1)
        idf=math.log((meta['num_docs']-meta['doc_count'][w]+0.5)/(meta['doc_count'][w]+0.5))
        bidf=ictf*pidf*idf
        tf=((k1+1)*meta['btf'][w][document_id])/(k1*(1-b+b*meta['doc_size'][document_id]/meta['avg_dl'])+meta['btf'][w][document_id])
        qtf=((k3+1)*query_meta['query_term_weight'][w])/(k3+query_meta['query_term_weight'][w])
        
        res=bidf*tf*qtf
        
        return res
    
    def score(self,query_meta,document_id,meta):
        words=set(query_meta['query_words']) & meta['doc_unique_terms'][document_id]
        score=0
        for w in words:
            score+=self.score_one(w,query_meta,meta,document_id)
        return score

In [212]:
def get_query_meta(query):
    if query in query_logs:
        return query_logs[query]
    else:
        query_words=query.split()
        query_length=len(query_words)
        query_term_weight={}
        for qw in query_words:
            if qw not in query_term_weight:
                query_term_weight[qw]=1
            else:
                query_term_weight[qw]+=1

        query_meta={}
        query_meta['query_length']=query_length
        query_meta['query_term_weight']=query_term_weight
        query_meta['query_words']=query_words
        query_meta['query']=query
        query_meta['related_query']=[]
        query_logs[query]=query_meta
        return query_meta

In [288]:
import pandas as pd
def do_search(query,meta):
    d_scores={}
    query_meta=get_query_meta(query)
    print("query:",query_meta['query'])
    print("result:")
    for i in range(meta['num_docs']):      
        d_scores[i]=[ranker.score(query_meta,i,meta)]
    my_frame=pd.DataFrame.from_dict(d_scores)
    print(my_frame)
#     print("related query:")
#     RQ_len=5
#     if len(query_meta['related_query'])<5:
#         RQ_len=len(query_meta['related_query'])       
#     for i in range(RQ_len):
#         print(query_meta['related_query'][i], end=" ")
    return d_scores

In [289]:
query_logs={}

In [290]:
ranker=bm25_ctf()

In [291]:
query="hello baby"

In [292]:
d_scores=do_search(query,meta)

query: hello baby
result:
          0  1        2         3  4
0  0.370611  0  0.19381  2.297318  0


# add some feedback

In [249]:
def get_update_word_list(d_scores,threshold=3):
    word_list=[]
    for i in d_scores:
        if d_scores[i]>0.1:
            pass
        else:
            for w in meta['doc_unique_terms'][i]:
                important_score=meta['btf'][w][i]*(1/meta['doc_count'][w])
                if important_score>threshold:
                    word_list.append(w)
                    print(w,important_score)
    return word_list

In [213]:
# used for feedback
def query_meta_update(query_meta,word_lists,update_rate=1):
    query_meta['related_query'].append(query_meta['query'])
    query_meta['query']+=" * "
    for w in word_lists:
        query_meta['query_words'].append(w)
        if w not in query_meta['query_term_weight']:
            query_meta['query_term_weight'][w]=update_rate
        else:
            query_meta['query_term_weight'][w]+=update_rate
        query_meta['query_length']+=update_rate*len(word_lists)
        if w not in query_meta['query']:
            query_meta['query']+=str(w)+" "
    query_meta['query']+="* "
    return query_meta

In [259]:
word_list=get_update_word_list(d_scores)

good 4.0
2018 4.0
haha 4.0
luck 4.0


In [260]:
query_logs[query]=query_meta_update(get_query_meta(query),word_list)

In [261]:
d_scores=do_search(query,meta)

query: hello baby * good 2018 haha luck * 
result:
0 0.37061128130176607
1 0
2 0.1938100058015275
3 2.2973177085906604
4 9.189270834362642
related query:
hello baby 